<a href="https://colab.research.google.com/github/serg-kas/NN-archive/blob/main/%D0%90%D0%BF%D1%80%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B8%D0%B9_%D0%BA%D1%83%D1%80%D1%81_2020_Light_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_32_Pro_CarRacing_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PRO задание 2 


Выберите любую другую игру на платформе OpenAI Gym и обучите нейросеть решать задачу в новой игровой среде.


## CarRacing-v0

**Предсказываем ВЕРОЯТНОСТЬ положения органов управления машинки из набора 5 предустановок на основании observation**



### Настройка окружения Colab

**Код для понижения версий tensorflow до 1.15 и keras до 2.3**

In [ ]:
# Посмотреть какие сейчас версии
!pip show tensorflow  
!pip show keras  

Name: tensorflow
Version: 2.4.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: tensorflow-estimator, google-pasta, h5py, astunparse, six, absl-py, termcolor, wheel, flatbuffers, opt-einsum, numpy, protobuf, typing-extensions, wrapt, gast, tensorboard, grpcio, keras-preprocessing
Required-by: fancyimpute
Name: Keras
Version: 2.4.3
Summary: Deep Learning for humans
Home-page: https://github.com/keras-team/keras
Author: Francois Chollet
Author-email: francois.chollet@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: pyyaml, scipy, h5py, numpy
Required-by: textgenrnn, keras-vis, kapre, fancyimpute


In [ ]:
# понижаем версию tensorflow
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 32kB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 512kB 62.0MB/s 
     |████████████████████████████████| 3.8MB 48.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=d1cbb7fdb7653711770d9233a9695d587230b5c2d7d96d05535c9cf86cab9d8b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [ ]:
# понижаем версию keras
!pip install keras==2.3

     |████████████████████████████████| 378kB 9.1MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


**Установка и настройка движка box2d и пакетов для показа видео в колабе**

In [ ]:
# Окружение из пакета Stable Baseline для OpenAI
# (код, представленный в документации разработчика)

# cmake - аналог GNU Make, но с другим синтаксисом, 
# libopenmpi-dev распределённые вычисления в кластере для дебиан, 
# zlib1g-dev - сжатие траффика
!apt install swig cmake libopenmpi-dev zlib1g-dev 
# устанавливаем непосредственно сам stable baseline и движок box2d
!pip install stable-baselines==2.5.1 box2d box2d-kengz 

# если убрать " > /dev/null 2>&1" то работа сервера будет выведена на экран, а этой командой перемещаем все в фоновый режим
!pip install gym pyvirtualdisplay > /dev/null 2>&1    # установка пакета для рендера видео
!pip install xvfbwrapper                              # чтобы воспроизводить видео в ячейке колаба  
!apt-get update                                       # обновляем все пакеты до новейшей версии
!sudo apt-get install xvfb                            # установка удаленного сервера, видео будет воспроизводиться в ячейке колаба
!apt-get install xdpyinfo                             # пакет для вывода процессов, происходящих на сервере

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libopenmpi-dev is already the newest version (2.1.1-8).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (765 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 146456 files and direct

### Импорт библиотек и подключение диска

In [ ]:
import gym                                          # здесь лежат все модели 
from gym import logger as gymlogger                 # загрузим библиотеку для изменения параметра вывода ошибок
from gym.wrappers import Monitor                    # библиотека для обертки видео из хранилища в ячейку
gymlogger.set_level(40)                             # для правильного отображения ошибок

import numpy as np                                  # библиотека массивов numpy
import random                                       # библиотека для генерации случайных значений

import keras                                       
from keras.models import Model, load_model          # из кераса подгружаем абстрактный класс базовой модели, метод загрузки модели
from keras.layers import Dense, Flatten, Input      # слои
from keras.layers import Lambda, Conv2D, Reshape    # слои
from keras.layers import MaxPooling2D, Multiply     # слои
from keras.layers import Dropout, Activation        # слои
from keras.layers import BatchNormalization         # слои
from keras.layers import Concatenate

from keras.optimizers import RMSprop, Adam          # оптимизаторы
from tensorflow.keras.utils import plot_model 
from tensorflow.keras import backend as K          

import time                                         # библиотека для расчета времени обучения
import io                                           # библиотека для работы с потоковыми данными

import matplotlib                                   # библиотека для визуализации процессов
import matplotlib.pyplot as plt                     # библиотека для построения графика
%matplotlib inline

Using TensorFlow backend.


In [ ]:
# расширение для преобразования в формат base64 (универсальный формат хранения сырых изображений в виде набора электрических сигналов)
import glob                                         # расширение для использования Unix обозначений при задании пути к файлу
import base64                                      
from IPython import display as ipythondisplay       # для работы с "сырым" форматом (набор сигналов, а не пиксели)
from IPython.display import HTML                    # библиотека для кодирования в код HTML
from pyvirtualdisplay import Display                # для создания окна дисплея

# Создаем объект дисплей, таким образом, симуляцию можно будет наблюдать в колабе
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Функции

#### Получение скорости

In [ ]:
##########################################################
# Функция получить скорость из синей полосы на фрейме
##########################################################
def get_speed(Frame):
  BLUE = np.array([0, 0, 255], dtype=np.float16)
  speedometer = Frame[93:83:-1, 19]

  speed = 0
  for i in speedometer:
    if np.array_equal(i, BLUE):
      speed+=1
    else:
      #print(speed)
      return speed

#### Предобработка данных

In [ ]:
###############################################################
# Готовая функция предобработки данных для подачи в нейросеть
# Подаем в функцию новый и предыдущий кадр
###############################################################

def preprocessFrames(newFrame,lastFrame):        

  nFrame = newFrame.astype(np.int32)      # переводим в целочисленный тип новый кадр
  # Цвета меняем в 1 канале 
  # Зеленый фон приводим к цвету 204 (сейчас смесь 204 и 230)
  nFrame[:,:,1][nFrame[:,:,1]==230] = 204
  # Серую дорогу приводим к цвету 207 (сейчас смесь 102, 105 и 107)
  nFrame[:,:,1][nFrame[:,:,1]==102] = 207
  nFrame[:,:,1][nFrame[:,:,1]==105] = 207
  nFrame[:,:,1][nFrame[:,:,1]==107] = 207
  # Белые и красные отбойники приводим к цвету фона (204)
  nFrame[:,:,1][nFrame[:,:,0]==255] = 204
  # Машинку приводим к цвету 100
  # Цвет 76 - серые пятнышки у колес 
  # (сами колеса сразу черные)
  nFrame[:,:,1][nFrame[:,:,0]==76] = 100              
  nFrame[:,:,1][nFrame[:,:,0]==204] = 100


  lFrame = lastFrame.astype(np.int32)     # переводим в целочисленный тип предыдущий кадр
  # Цвета меняем в 1 канале 
  # Зеленый фон приводим к цвету 204 (сейчас смесь 204 и 230)
  lFrame[:,:,1][lFrame[:,:,1]==230] = 204
  # Серую дорогу приводим к цвету 105 (сейчас смесь 102, 105 и 107)
  lFrame[:,:,1][lFrame[:,:,1]==102] = 105
  lFrame[:,:,1][lFrame[:,:,1]==107] = 105
  # Белые и красные отбойники приводим к цвету фона (204)
  lFrame[:,:,1][lFrame[:,:,0]==255] = 204
  # Машинку сделаем черной (код 0). 
  # Цвет 76 - серые пятнышки у колес 
  # (сами колеса сразу черные)
  lFrame[:,:,1][lFrame[:,:,0]==76] = 50              
  lFrame[:,:,1][lFrame[:,:,0]==204] = 50

  # НА РАЗНИЦЕ ФРЕЙМОВ БУДЕТ НУЛЕВОЙ ФОН, ДОРОГА ЦВЕТА 207-105=102 И МАШИНКА ЦВЕТА 100-50=50

  # Задаём разницу между новым и предыдущим кадром   
  deltaFrame = nFrame - lFrame   

  # Отрезаем низ экрана с черной полосой (12 пикселей)
  # и по полоске с боков (по 6 пикселей)
  deltaFrame = deltaFrame[0:84, 6:90]

  # Берем только канал 1
  # (переходим к монохрому)
  deltaFrame = deltaFrame[:,:, 1]

  #plt.figure(figsize=(5, 5))
  #plt.imshow(deltaFrame, plt.cm.gray) 
  #plt.show() 

  # МАСШТАБИРОВАНИЕ ВОЗМОЖНО УБЕРЕМ, ПОСТАВИВ НА ВХОД НОРМАЛИЗАЦИЮ
  # Масштабирование чисел от 0 до 1 
  maxValue = deltaFrame.max() if deltaFrame.max()> abs(deltaFrame.min()) else abs(deltaFrame.min())
  if maxValue != 0:
      deltaFrame=deltaFrame/maxValue

  # функция вернет разницу между кадрами
  return deltaFrame                       

#### Функция потерь в соответствии с вознаграждением



In [ ]:
############################################################################
# Функция потерь в керасе имеет вид def loss(yTrue,yPred)
# ...А так как нам нужно включить reward в loss, создаем
# ...функцию rewardedLoss поверх неё, чтобы на входе добавить episodeReward
############################################################################

def rewardedLoss(episodeReward): # на вход принимаем episodeReward (награда)

  def loss(yTrue,yPred):
    # yTrue - это фактически сделанное действие в формате OHE, размерностью (5,) 
    # yPred - это выход сетки - вектор размерностью (5,)   

    # вариант из вебинара
    #tmpPred = Lambda(lambda x: keras.backend.clip(x,0.05,0.95))(yPred)
    #tmpLoss = Lambda(lambda x:-yTrue*keras.backend.log(x) - (1-yTrue)*(keras.backend.log(1-x)))(tmpPred)

    # нагуглил как вариант бинарной кроссэнтропии
    #tmpLoss = K.max(yPred,0)-yPred * yTrue + K.log(1+K.exp((-1)*K.abs(yPred)))

    # классический вариант из гитхаба
    tmpPred = Lambda(lambda x: K.clip(x,1e-10,1.0))(yPred)
    tmpLoss = Lambda(lambda x:-yTrue*K.log(x))(tmpPred)

    # обновленная функция потерь - "функция политики"
    policyLoss = Multiply()([tmpLoss, episodeReward])      #добавляем в loss умножение на награду за эпизод

    return policyLoss                                      # ввели обновленную функцию политики

  return loss                                              # возвращаем обновленную функцию политики

#### Функция генерации игрового эпизода с участием нейросети

In [ ]:
##################################################################################
# Создадим функцию, которая будет генерировать игровой эпизод с участием нейросети
##################################################################################
# игра начнется с того, что в новом эпизоде наш агент сделает движение(actualAction), которое предсказала наша...
# ...сетка policyNetwork, принявшая на вход состояние среды reshapedInput с подбитой под сеть формой. 
# reshapedInput же получили из состояния processedNetworkInput, которое представляет собой разницу в кадрах между новым и предыдущим...
# ...кадром, полученную функцией preprocessFrames.
 
 
def generateEpisode(policyNetwork):               # подаем на вход функции модель нейросети

  statesList = []                                 # список состояний в течение эпизода, размер = (x,84,84)
  actionList=[]                                   # список действий для подачи в окружение
  rewardsList=[]                                  # список наград за каждое действие
  speedsList=[]                                   # СПИСОК СКОРОСТЕЙ
  networkOutputList=[]                            # список выходов нейросети

  env=gym.make('CarRacing-v0')                    # cоздали среду
  observation = env.reset()                       # перезагрузили состояние среды
  newObservation = observation                    # на первом шаге новое состояние берем из текущего
  done = False                                    # игровой эпизод активен(не завершён)

  # ПРЕДУСТАНОВКИ ДЛЯ ПЕРЕДАЧИ В ОКРУЖЕНИЕ
  envActionList = [[-1.0, 0.0, 0.0],              # Поворот налево
                   [+1.0, 0.0, 0.0],              # Поворот направо
                   [ 0.0, 0.0, 0.8],              # Томожение
                   [ 0.0, 1.0, 0.0],              # Ускорение
                   [ 0.0, 0.0, 0.0]]              # Ничего не делаем
                      

 
  while done == False:                            # пока игровой эпизод не завершён

    # На вход сети будет подаваться очередное состояние - разница между кадрами. 
    processedNetworkInput = preprocessFrames(newFrame=newObservation, lastFrame=observation)   

    statesList.append(processedNetworkInput)      # добавим в список состояний (впоследствие станет 'x'ом для входа в нейронку)
  
    # Размер 'x' - (84,84), делаем размерность (84,84,1))
    reshapedInput = np.expand_dims(processedNetworkInput, axis=2)
    # Размер 'x' - (84,84,1), делаем размерность (1,84,84,1))
    reshapedInput = np.expand_dims(reshapedInput, axis=0)    
        

    # КАЖДОЕ ЗНАЧЕНИЕ ВЕКТОРА ПРЕДИКТА - ВЕРОЯТНОСТЬ ВЫБОРА ОДНОЙ ИЗ 5 ПРЕДУСТАНОВОК ОРГАНОВ УПРАВЛЕНИЯ
    # Получаем вектор предикта    
    actionProbability = policyNetwork.predict(reshapedInput, batch_size=1)[0]
    networkOutputList.append(actionProbability)   # добавляем вектор предикта к списку выходов нейронки
 
    # Определяем номер предустановки для передачи в окружение по вероятности
    #assert np.isnan(actionProbability).sum()==0, 'NaN в предикте'
    indexAction = np.random.choice(a=[0,1,2,3,4], size=1, p=actionProbability)[0]
    #print(actionProbability, indexAction)

    actualAction = np.array([0,0,0,0,0])
    actualAction[indexAction] = 1
    actionList.append(actualAction)   # добавляем выбранное действие в список actionList в формате OHE

    # текущий newObservation записываем как старый перед тем, как сделать следующий шаг
    observation = newObservation             
    # В ОКРУЖЕНИЕ ПОДАЛИ значение envAction по индексу indexAction
    newObservation, reward, done, info = env.step(np.array(envActionList[indexAction])) # сделали новый шаг, получили новую награду, новое состояние

    # Добавили текущую награду в список
    # ПРОБУЕМ ограничить ревард
    #reward = np.clip(reward, -1, 1)
    reward = np.clip(reward, a_min=-100.0, a_max=1.0)
    rewardsList.append(reward)            

    # Получили скорость и добавили в список
    speedsList.append(get_speed(newObservation))   
 
    if done:                                 # если игровой эпизод закончен
      break                                  # завершаем цикл
 
  env.close()                                # закрываем текущую среду
  return statesList, actionList, rewardsList, speedsList, networkOutputList
  # функция возвращает: список состояний в течение эпизода,список действий в течение эпизода, список наград за каждое действие,
  # список скоростей и список вероятностей выбора предустановок органов управления

#### Функция эффективного определения вознаграждения

In [ ]:
 ###########################
# Сформулируем функцию, которая распределит ненулевую награду для всех шагов в удачном/неудачном розыгрыше
###########################

def processRewards(rewardList): # подадим в функцию список наград
  rewardDecay = 0.99            # установим высший коэффициент награды для ближнего действия к победному/проигрышному действию
  tmpReward = 0                 # создадим временную переменную для награды
  rewardDecayed = np.zeros_like(rewardList,dtype=np.float32) # создадим массив из нулей для нового формата наград
  
  for i in range(len(rewardList)-1,-1,-1): # будем идти в обратную сторону от награды с шагом "-1"
    if rewardList[i] == 0:                 # если награда нулевая 
      tmpReward = tmpReward*rewardDecay    # зададим ей коэффициент
      rewardDecayed[i] = tmpReward         # и добавим в массив
    else:                                  # иначе
      tmpReward = rewardList[i]            # оставим награду неизменной
      rewardDecayed[i] = tmpReward         # и введём в том же виде в массив
  
  # запустим нормализацию значений наград, что позволит сетке лучше понимать, где были хорошие и плохие шаги
  rewardDecayed -= np.mean(rewardDecayed)  # вычтем среднее
  rewardDecayed /= np.std(rewardDecayed)   # разделим на стандартное отклонение

  #maxReward = rewardDecayed.max()           # Нормализация к 0 1 
  #if maxReward != 0:
  #  rewardDecayed = rewardDecayed / maxReward 

  
  return rewardDecayed
  # функция возвращает обновленный формат наград

#### Функция генерации игрового эпизода с последующей тренировкой сети


In [ ]:
##################################################
# Теперь создадим серию игровых эпизодов
# Затем предобработаем данные и скормим нейросети
##################################################
def generateEpisodeBatchesTraining(model, nBatches=10): # подаем на вход заданное количество игровых серий
  env = gym.make('CarRacing-v0')                        # создаем игровую среду 
  batchStateList = []                                   # зададим серию списков состояний
  batchActionList = []                                  # зададим серию списков движений
  batchRewardsList = []                                 # зададим серию списков наград
  batchSpeedsList = []                                  # зададим серию СПИСКОВ СКОРОСТЕЙ
  batchNetworkOutputList = []                           # зададим серию списков из предсказаний нейронки

  for i in range(nBatches):                             # для каждой серии 
    statesList, actionsList, rewardsList, speedsList, networkOutputList = generateEpisode(model) # сгенерируем игровой эпизод

    batchStateList.extend(statesList)                   # добавим список состояний в серию списков состояний
    batchNetworkOutputList.extend(networkOutputList)    # добавим список вероятностей в серию списков из вероятности идти вверх
    batchActionList.extend(actionsList)                 # добавим список движений в серию списков движений
    batchSpeedsList.extend(speedsList)                    # добавим список движений в серию СПИСКОВ СКОРОСТЕЙ
    batchRewardsList.extend(rewardsList)                # добавим список наград в серию списков наград
 
  episodeReward = np.expand_dims(processRewards(batchRewardsList), 1) # зададим награды и изменим форму массива с добавлением оси
  
  
  episodeSpeed = np.array(batchSpeedsList)               
  #episodeSpeed -= np.mean(episodeSpeed)                 # вычтем среднее
  #episodeSpeed /= np.std(episodeSpeed)                  # разделим на стандартное отклонение
  maxSpeed = episodeSpeed.max()                         # Пересчитаем в диапазон 0 1
  if maxSpeed != 0:
    episodeSpeed = episodeSpeed / maxSpeed 
  episodeSpeed = np.expand_dims(episodeSpeed, 1)        # добавим ось 

  x = np.array(batchStateList)                          # сформируем массив из серии списков состояний в качестве 'x' для нейросети
  x = np.expand_dims(x, 3)

  #yTmp = np.array(batchActionList)                      # зададим 'y' как серию из списков движений 
  #yTrue = np.expand_dims(yTmp, 1)                       # подгоним форму 'y' массива с добавлением оси
  yTrue = np.array(batchActionList) 

  #print(x.shape)
  #print(episodeReward.shape)
  #print(yTrue.shape)

  #history = policyNetworkTrain.fit(x=[x,episodeReward,episodeSpeed], y=yTrue, epochs=10, verbose=0) # скормим нейросети серию 'x'ов и 'y'ов БЫЛО 5 epochs
  history = policyNetworkTrain.fit(x=[x,episodeReward], y=yTrue, epochs=10, verbose=0) # скормим нейросети серию 'x'ов и 'y'ов БЫЛО 5 epochs
  
  batchLoss = history.history['loss'][-1]
  return batchStateList, batchActionList, batchRewardsList, batchSpeedsList, batchNetworkOutputList, batchLoss
  # функция вернёт серию списков состояний, серию списков движений, серию списков наград, серию списков из вероятности идти вверх

### Модели



#### Сверточная модель

Выход на 5 нейронов с активацией softmax

In [ ]:
inputs = Input(shape=(84,84,1))     

x = Conv2D(64, (5, 5), strides=(2,2), name='conv1')(inputs)    
x = Activation('relu')(x)   
x = Conv2D(128, (5, 5), strides=(2,2), name='conv2')(x)  
x = Activation('relu')(x)   
x = MaxPooling2D()(x)
x = BatchNormalization()(x)

x = Conv2D(256, (3, 3), strides=(1,1), name='conv3')(x)    
x = Activation('relu')(x)   
x = Conv2D(512, (3, 3), strides=(1,1), name='conv4')(x)  
x = Activation('relu')(x)   
x = MaxPooling2D()(x)
x = BatchNormalization()(x)

conv_block = Flatten()(x)

inp_transit = Flatten()(inputs)
inp_transit_norm = BatchNormalization()(inp_transit)

to_dense = Concatenate()([inp_transit_norm,conv_block])

x = Dense(1024, activation='relu')(to_dense) 
x = BatchNormalization()(x)
x = Dropout(0.2)(x)                   #Было 0.5

x = Dense(128, activation='relu')(x) 
x = BatchNormalization()(x)
x = Dropout(0.2)(x)                   #Было 0.5

modelOutput = Dense(5, activation='softmax')(x)

policyNetworkModel = Model(inputs=inputs, outputs=modelOutput)              
policyNetworkModel.summary()

#plot_model(policyNetworkModel) 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 84, 84, 1)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 40, 40, 64)   1664        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 40, 40, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 18, 18, 128)  204928      activation_1[0][0]               
__________

#### Модель для обучения

In [ ]:
####################################################
# Создаем керас-оптимизатор и нейронку для обучения
####################################################

episodeReward = Input(shape=(1,), name='episodeReward')                             # задаем награду за эпизод

#episodeSpeed = Input(shape=(1,), name='episodeSpeed')                               # задаем СКОРОСТЬ

# задаем сеть с добавлением на вход НАГРАДЫ и СКОРОСТИ
#policyNetworkTrain = Model(inputs=[inputs, episodeReward, episodeSpeed],outputs=modelOutput)   
policyNetworkTrain = Model(inputs=[inputs, episodeReward],outputs=modelOutput) 
 
#plot_model(policyNetworkTrain) 

In [ ]:
#myOptimizer = RMSprop(lr=0.00010)                                                    # выбрали оптимизатор с заданной скоростью обучения
#myOptimizer = Adam(lr=0.00300)
#myOptimizer = Adam(lr=0.00025)
myOptimizer = Adam(lr=0.00010)

# задаем сеть с новой функцией потерь policy
policyNetworkTrain.compile(optimizer=myOptimizer, loss=rewardedLoss(episodeReward)) 
#policyNetworkTrain.compile(optimizer=myOptimizer, loss=rewardSpeedLoss(episodeReward, episodeSpeed))


### **Обучаем сеть на серии игровых эпизодов**

In [ ]:
PATH = '/content/drive/My Drive/my_sample_data/CarRacing-v0/'

policyNetworkModel = load_model(PATH+'policyNetworkModel.h5')

trainingTimes = 151                   # установим количество тренировок 
for training in range(trainingTimes): # для каждой тренировки
  startTime = time.time()
  # сделаем по 10 игровых эпизодов(1 эпоха - 10 игр)
  statesList,ActionList,rewardsList, speedsList, networkOutputList, batchLoss = generateEpisodeBatchesTraining(policyNetworkModel, 10) 
  endTime = time.time()
  print("Тренировка = " + str((training)+1))                           # выведем на экран номер очередной тренировки
  print("Время тренировки = " + str(round(endTime - startTime))+"сек") # время тренировки
  print("Ошибка на тренировке = " + str(round(batchLoss, 5)))          # выведем на экран ошибку на тренировке
  rr=np.array(rewardsList)                                             # сформируем все награды
  # выведем на экран сумму выигранных и проигранных очков
  print("Выиграли очков = "+ str(len(rr[rr>0])) + " Проиграли очков = " + str(len(rr[rr<0]))) 
  # выведем среднюю и максимальную скорость на эпизоде
  ss=np.array(speedsList)
  print('Средняя скорость', ss.mean(), 'Максимальная скорость', ss.max())
  print("")
  if training % 10 == 0:                                                     # через каждые 10 эпизодов
    policyNetworkModel.save(PATH+'policyNetworkModel.h5')                    # сохраним модель
    policyNetworkModel.save(PATH+'policyNetworkModel' + str(training)+'.h5') # и сохраним модель с количеством тренировок в названии
    with open(PATH+'rewardsModelSimple.txt','a') as recordingRewards:        # в текстовом файле будем вести записи с динамикой выигранных очков
      recordingRewards.write("training = " + str(training) + '  выигранных очков = ' + str(len(rr[rr > 0])))
      recordingRewards.write("\n")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Track generation: 1136..1424 -> 288-tiles track

Track generation: 1120..1414 -> 294-tiles track
Track generation: 1095..1373 -> 278-tiles track
Track generation: 1283..1608 -> 325-tiles track
Track generation: 1238..1552 -> 314-tiles track
Track generation: 1140..1429 -> 289-tiles track
Track generation: 1187..1488 -> 301-tiles track
Track generation: 1250..1568 -> 318-tiles track
Track generation: 1200..1504 -> 304-tiles track
Track generation: 1102..1382 -> 280-tiles track
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Тренировка = 1
Время тренировки = 124сек
Ошибка на тренировке = -0.50723
Выиграли очков = 153 Проиграли очков = 9847
Средняя скорость 0.1336 Максимальная скорость 3

Track generation: 1013..1275 -> 262-tiles track
Track generation: 1219..1528 -> 309-tiles track
Track generation: 1117..1403 -> 286-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1173..1470 -> 297

### Сыграть эпизод обученной моделью и воспроизвести видео


In [ ]:
################################################
# Задаем функции записи и воспроизведения видео
################################################
def wrapEnv(env):                           # зададим функцию-обёртку над средой
  env = Monitor(env, './video', force=True) # класс Monitor будет записывать игру в видеофайл
  return env                                # теперь вызов среды через функцию wrapEnv будет запускать видеозапись
 
def showVideo():                            # функция для воспроизведения видео в ноутбуке
  mp4list = glob.glob('video/*.mp4')        # возвращает список путей к видео по этому шаблону
  if len(mp4list) > 0:                      # если по этому пути нашелся хотя бы один файл
    mp4 = mp4list[0]                        # то берём самый свежий файл
    video = open(mp4, 'r+b').read()         # открываем в режиме чтения/записи бинарного файла
    encoded = base64.b64encode(video)       # кодируем видео в 64-разрядный код 
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    # Декодируем ascii формат в видео mp4 - "format(encoded.decode('ascii'))))"
    # для воспроизведения на разных браузерах - "<source src="data:video/mp4"
    # задаём высоту экрана 400px - "height: 400px;"
    # активируем элементы управления в видео(типа кнопки паузы и т.п) - "controls"
    # включаем автоповтор после окончания воспроизведения - "loop"
    # видео автоматически воспроизведется по готовности - "autoplay"
    # текстовая альтернатива описывающая объект в случае невозможности отображения - "alt="test""
    
  else:                           # если путь не нашелся 
    print("Could not find video") # то выведем на печать, что не смог найти видео

In [ ]:
#########################################
# Задаем функцию сыграть и показать игру
#########################################
def playAndShowEpisode(policyNetwork):           # подаем на вход функции модель нейросети
    env = wrapEnv(gym.make('CarRacing-v0'))      # создаем среду в режиме записи
    done = False                                 # игровой эпизод не завершён
    observation = env.reset()                    # задаем начальное состояние среды, которое наблюдает агент
    newObservation = observation                 # задали новое состояние которое наблюдает агент из текущего

    # ПРЕДУСТАНОВКИ ДЛЯ ПЕРЕДАЧИ В ОКРУЖЕНИЕ
    envActionList = [[-1.0, 0.0, 0.0],           # Поворот налево
                     [+1.0, 0.0, 0.0],           # Поворот направо
                     [ 0.0, 0.0, 0.8],           # Томожение
                     [ 0.0, 1.0, 0.0],           # Ускорение
                     [ 0.0, 0.0, 0.0]]           # Ничего не делаем
                     

    while done == False:                         # пока игровой эпизод не завершён
        # На вход сети будет подаваться очередное состояние - разница между кадрами. 
        processedNetworkInput = preprocessFrames(newFrame=newObservation,lastFrame=observation) # зададим это состояние
        # Размер 'x' - (84,84), делаем размерность (84,84,1))
        reshapedInput = np.expand_dims(processedNetworkInput, axis=2)
        # Размер 'x' - (84,84,1), делаем размерность (1,84,84,1))
        reshapedInput = np.expand_dims(reshapedInput, axis=0)  

        # Получаем вектор предикта 
        actionProbability = policyNetwork.predict(reshapedInput, batch_size=1)[0]
    
        # Определяем номер предустановки для передачи в окружение по вероятности
        indexAction = np.random.choice(a=[0,1,2,3,4], size=1, p=actionProbability)[0]
        

        env.render()                   # запускаем воспроизведение среды
        observation = newObservation   # текущий newObservation записываем как старый, перед тем как сделать следующий шаг
        # В ОКРУЖЕНИЕ ПОДАЛИ envAction по индексу indexAction
        newObservation, reward, done, info = env.step(np.array(envActionList[indexAction])) # сделали новый шаг, получили новую награду, новое состояние
 
    env.close() # закрываем игровую среду после окончания игры
    showVideo() # показать видео игры

In [ ]:
# Взглянем как сыграет обученная сеть
# Подгружаем модель из сохранённых
policyNetworkModel99 = load_model('/content/drive/My Drive/my_sample_data/CarRacing-v0/policyNetworkModel.h5') 
# и запускаем игровой эпизод
playAndShowEpisode(policyNetworkModel99)                     

**Удалось подобрать совокупность параметров, при которых машинка более менее поехала и начала обучаться.** 

**Какой бы метод reinforcement learning не применялся (и более продвинутый чем использованный мной)- источники говорят о нескольких десятках (сотнях) тысяч тренировок до вероятного прохождения трассы.**
